In [ ]:
#https://www.liip.ch/en/blog/sentiment-detection-with-keras-word-embeddings-and-lstm-deep-learning-networks

In [21]:
import keras as keras

In [22]:
from keras.datasets import imdb
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [23]:
from numpy import array
from keras.preprocessing.text import one_hot

docs = [
    'Gut gemacht',
    'Gute arbeit',
    'Super idee',
    'Perfekt erledigt',
    'exzellent',
    'naja',
    'Schwache arbeit.',
    'Nicht gut',
    'Miese arbeit.',
    'Hätte es besser machen können.'
]

vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[1, 45], [39, 30], [49, 22], [48, 20], [24], [14], [31, 30], [5, 1], [1, 30], [29, 2, 30, 7, 45]]


In [24]:
# set all inputs to fix size, add padding to those smaller than selected size.
from keras.preprocessing import sequence
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [25]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [10]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [28]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedding_vector_length = 256 #32
hidden_dimensions = 1024
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 256)          1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               142800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,422,901
Trainable params: 1,422,901
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=64) #nb_epoch=3, later.

/Users/danielobembe/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 559s 22ms/step - loss: 0.4309 - acc: 0.8046 - val_loss: 0.3875 - val_acc: 0.8375


In [30]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.75%


In [31]:
bad = "this movie was terrible and bad"
good = "i really liked the movie and had fun"
for review in [good,bad]:
    tmp = []
    for word in review.split(" "):
        tmp.append(word_to_id[word])
    tmp_padded = sequence.pad_sequences([tmp], maxlen=max_review_length) 
    print("%s. Sentiment: %s" % (review,model.predict(array([tmp_padded][0]))[0][0]))

i really liked the movie and had fun. Sentiment: 0.65454674
this movie was terrible and bad. Sentiment: 0.7258723
